In [47]:
# Ejercicio 2
from numpy import sqrt, log
from numpy.random import uniform
import math
from numpy import vectorize

# a)
print("a)")
data = [1.628, 1.352, 1.8, 1.420, 1.594, 2.132, 1.614, 1.924, 1.692]


def sample_mean(data):
    summation = 0
    n = len(data)

    for x in data:
        summation += x

    return summation / n


def sample_stdev(data, mean_estimation):
    summation = 0
    n = len(data)

    for x in data:
        summation += (x - mean_estimation) ** 2

    return sqrt(summation / (n - 1))


mean_estimation = sample_mean(data)
stdev_estimation = sample_stdev(data, mean_estimation)

print(f"Estimación de la media: {mean_estimation}")
print(f"Estimación de la desviación estándar: {stdev_estimation}")

# b)
print("b)")

def normal_cdf(x, mu, stdev):
    return math.erf((x - mu) / (stdev * math.sqrt(2))) / 2 + 0.5

normal_cdf = vectorize(normal_cdf)

def reject_normal(mu, sigma):
    while True:
        y2 = -log(1 - uniform())
        y = -log(1 - uniform())

        if y2 >= (y - 1) ** 2 / 2:
            if uniform() < 0.5:
                return y * sigma + mu
            else:
                return -y * sigma + mu

def ks_statistic(data, mu, stdev):
    n = len(data)
    d = 0
    sorted_sample = sorted(data)
    norm_cdf = normal_cdf(sorted_sample, mu=mu, stdev=stdev)

    for i in range(1, n + 1):
        d = max(d, i / n - norm_cdf[i - 1], norm_cdf[i - 1] - (i - 1) / n)
    return d

d_ks = ks_statistic(data, mean_estimation, stdev_estimation)

print(f"Valor del estadístico: {d_ks}")

# c)
print("c)")
# Estima mejor
def estimate_pvalue_realistic(data, d_ks, mu, stdev):
    pvalue_count = 0
    n = len(data)
    m = 1000
    
    for _ in range(m):
        # NOTE: NO OLVIDAR ORDENAR
        sample = sorted([reject_normal(mu, stdev) for _ in range(n)])
        sample_mean_estimation = sample_mean(sample)
        sample_stdev_estimation = sample_stdev(sample, sample_mean_estimation)

        pvalue_count += int(ks_statistic(sample, mu=sample_mean_estimation, stdev=sample_stdev_estimation) >= d_ks)

    pvalue_estimation = pvalue_count/m

    return pvalue_estimation

pvalue_estimation = estimate_pvalue_realistic(data, d_ks, mean_estimation, stdev_estimation)
print(f"Estimación del p-valor (más realista): {pvalue_estimation}")

# Estima un poco peor
def estimate_pvalue(data, d_ks):
    pvalue_count = 0
    n = len(data)
    m = 10000
    
    for _ in range(m):
        uniforms = uniform(0, 1, n)
        uniforms.sort()
       
        d_j = 0
        
        for j in range(n):
            u_j = uniforms[j]
            d_j = max(d_j, (j + 1)/n - u_j, u_j - j/n)
        
        if d_j >= d_ks:
            pvalue_count += 1
    
    pvalue_estimation = pvalue_count/m
    
    return pvalue_estimation

pvalue_estimation = estimate_pvalue(data, d_ks)
print(f"Estimación del p-valor: {pvalue_estimation}")

# d)
print("d)")
print("No hay suficiente evidencia para rechazar la hipótesis nula")

# e)
print("e)")
n = len(data)
us = [0.23, 0.12, 0.45, 0.67, 0.01, 0.51, 0.38, 0.92, 0.84]
us.sort()
       
d_j = 0

for j in range(n):
    u_j = us[j]
    d_j = max(d_j, (j + 1)/n - u_j, u_j - j/n)

print(f"Valor del estadístico original: {d_ks}")
print(f"Valor obtenido a partir de las uniformes: {d_j}")
print("De donde se deduce que el obtenido para estos datos es mayor que el original")

a)
Estimación de la media: 1.6840000000000002
Estimación de la desviación estándar: 0.2418367217773182
b)
Valor del estadístico: 0.15347199452116866
c)
Estimación del p-valor (más realista): 0.788
Estimación del p-valor: 0.9653
d)
No hay suficiente evidencia para rechazar la hipótesis nula
e)
Valor del estadístico original: 0.15347199452116866
Valor obtenido a partir de las uniformes: 0.15666666666666662
De donde se deduce que el obtenido para estos datos es mayor que el original


In [172]:
# Ejercicio 3
from numpy.random import uniform
from numpy import bincount

def binom_probs(n, p):
    p0 = (1-p)**n
    ps = [p0]

    for j in range(1, n + 1):
        p0 *= ((n-j+1)/j) * (p / (1-p))
        ps.append(p0)

    return ps

def binom_prob_from_frecs(n, frecuencies):
    '''
    Sirve para estimar parámetro "p" de una distribución binomial a partir del n y sus frecuencias.
    '''
    m = sum(frecuencies)

    return sum(i*x for i, x in enumerate(frecuencies))/(n*m)

def optimized_bin(n, p):
    p0 = (1-p)**n
    F = p0
    
    max_bin = int(n * max(p, (1-p)))
    
    for j in range(1, max_bin + 1):
        p0 *= ((n-j+1)/j) * (p / (1-p))
        F += p0

    u = uniform()
    if (u >= F):
        j = max_bin + 1
        
        while(u >= F):
            p0 *= ((n-j+1)/j) * (p / (1-p))
            F += p0
            j += 1
            
        return j-1
    else:
        j = max_bin
        
        while(u < F):
            F -= p0
            p0 *= j/((n-j+1) * (p / (1-p)))
            j -= 1
            
        return j+1

# a)
print("a)")
Ns = [490, 384, 111, 15]
m = sum(Ns)

n = 3
p_estimation = sum(i*x for i, x in enumerate(Ns))/(n*m)
print(f"Estimación del parámetro p: {p_estimation}")
ps = binom_probs(n, p_estimation)

# b)
print("b)")

# NOTE: m fue definida más arriba
def pearson_statistic(Ns, ps):
    return sum( (Ni - m*pi)**2/(m * pi) for Ni, pi in zip(Ns, ps))

t = pearson_statistic(Ns, ps)

print(f"Valor del estadístico bajo la muestra original: {t}")

# d)
print("d)")

# NOTE: m fue definida más arriba
# Versión naïve
def estimate_pvalue(t, n, p, num_of_sims=10):
    assertions = 0

    for _ in range(num_of_sims):
        sample = [optimized_bin(n, p) for _ in range(m)]
        Ms = bincount(sample)

        p_sim = binom_prob_from_frecs(n, Ms)
        ps_obs = binom_probs(n, p_sim)
        
        t_obs = pearson_statistic(Ms, ps_obs)
        
        assertions += int(t_obs >= t)
    
    return assertions/num_of_sims 

pvalue_estimation = estimate_pvalue(t, n, p_estimation, 200)
print(f"Estimación del p-valor: {pvalue_estimation}")

# Versión naïve pero no tanto
# def estimate_pvalue_frecs(t, n, p, num_of_sims=10):
#     assertions = 0

#     for _ in range(num_of_sims):
#         Ms = []
#         remaining_n = m
#         cummulated_p = 0
        
#         # for p in ps:
#         #     from scipy.stats import binom
#         #     # NOTE: Sé que esto se podría optimizar para que no se evalúe la última binomial
            
#         #     sample = binom(remaining_n, current_p).rvs()
#         #     Ms.append(sample)

#         #     remaining_n -= sample
#         #     cummulated_p += p

#         # t0 = pearson_statistic(Ms, ps)
#         # assertions += int(t0 >= t)
        
#         #sample = [optimized_bin(n, p) for _ in range(m)]
#         Ms = []
#         remaining = m
#         cummulated_p = 0

#         for p in ps:
#             # NOTE: Sé que esto se podría optimizar para que no se evalúe la última binomial
#             if 1-cummulated_p > 0:
#                 current_p = min(p/(1 - cummulated_p), 1)
#                 #print(f"p: {1-cummulated_p}")
#                 from scipy.stats import binom
#                 #sample = binom(remaining, current_p).rvs()
#                 sample = optimized_bin(remaining, current_p)
#             else:
#                 sample = remaining
#             Ms.append(sample)

#             remaining -= sample
#             cummulated_p += p

#         p_sim = binom_prob_from_frecs(n, Ms)
#         ps_obs = binom_probs(n, p_sim)
        
#         t_obs = pearson_statistic(Ms, ps_obs)
        
#         assertions += int(t_obs >= t)
    
#     return assertions/num_of_sims 

# pvalue_estimation = estimate_pvalue_frecs(t, n, p_estimation, 200)
# print(f"Estimación del p-valor (generando las frecuencias): {pvalue_estimation}")

from scipy.stats import multinomial

# NOTE: Utiliza los valores de n y m de más arriba
def pvalue_estimation2(t, ps, num_of_sims):
    assertions = 0

    samples = multinomial(m, ps).rvs(num_of_sims)
    
    for Ms in samples:
        p_sim = binom_prob_from_frecs(n, Ms)
        ps_obs = binom_probs(n, p_sim)
        
        t_obs = pearson_statistic(Ms, ps_obs)
        assertions += int(t_obs >= t)

    return assertions/num_of_sims

p2 = pvalue_estimation2(t, ps, 200)
print(f"Estimación del p-valor (usando multinomial): {p2}")


a)
Estimación del parámetro p: 0.217
b)
Valor del estadístico bajo la muestra original: 3.018118522843664
d)
Estimación del p-valor: 0.28
Estimación del p-valor (usando multinomial): 0.27


In [6]:
# Ejercicio 4
from numpy.random import uniform
from scipy.stats import norm
from numpy import sqrt

def g(u):
    return (1/u - 1)/(u**2 * (1 + (1/u - 1)**4))

def update_mean(oldmean, newdata, n):
    return oldmean + (newdata - oldmean) / n

def update_scuad(oldsquad, mean, oldmean, n):
    return oldsquad * (1 - 1 / (n - 1)) + n * (mean - oldmean) ** 2

def montecarlo(fun, sl=0.001, alpha=0.05, checkpoints={}):
    mean = fun(uniform())
    percentil = norm.ppf(1-alpha/2, 0, 1)
    squad = 0
    n = 1
    
    # NOTE: Esto verifica unicamente el SEMI-ancho
    while n < 100 or sqrt(squad/n) * percentil >= sl:
        n += 1
        
        x = fun(uniform())
        
        oldmean = mean
        mean = update_mean(oldmean, x, n)
        squad = update_scuad(squad, mean, oldmean, n)
        
        if n in checkpoints.keys():
            checkpoints[n] = [mean, sqrt(squad), (mean - sl, mean + sl)]

    return mean, {n: [mean, sqrt(squad), (mean - sl, mean + sl)]}


import pandas as pd

checkpoints = {1000: None, 5000: None, 7000: None}
estimation, last_results = montecarlo(g, sl=0.001, alpha=0.05, checkpoints=checkpoints)

def setup_df(analysis_dict):
    cols = ['I', 'S', 'IC']
    df = pd.DataFrame.from_dict(analysis_dict, orient='index', columns=cols)

    return df

def pretty_print_analysis(df):
    df_copy = df.copy()
    df_copy = df_copy.round(4)
    df_copy['IC'] = df_copy['IC'].apply(lambda t: (round(t[0], 4), round(t[1], 4)))
    
    print(df_copy)

df = setup_df(checkpoints)
pretty_print_analysis(df)

           I       S                IC
1000  0.8022  0.6554  (0.8012, 0.8032)
5000  0.7821  0.6560  (0.7811, 0.7831)
7000  0.7913  0.6595  (0.7903, 0.7923)


In [6]:
import sympy as sp

sl = 0.001
alpha = 0.05
sample = [montecarlo(g, sl=sl, alpha=alpha) for _ in range(10)]

from statutils.analysis import probability_of

def fun(x):
    return x/(1 + x**4)

x = sp.Symbol("x")

Ival = float(sp.Integral(fun(x),(x, 0, sp.oo)).evalf())

print(probability_of(sample, lambda x: (abs(x[0] - Ival) < sl )))
print(f"Expected: {1-alpha}")

1
Expected: 0.95
